# Stochastic Linear Programming and solving in Julia

In this notebook, emphasis is placed on stochastic linear programming and how to solve such optimization problems using Julia. 

We first give a very compact and high-level introduction to stochastic linear programming. The topic was covered more extensively in the related lectures (Stochastic Linear Programming 1 and 2) and related hands-on sessions. After recalling those basics, we dive into a number of example problems that can formulated as stochastic linear program. We go from a text-based description of those problems, to their mathematical formulation, and finally to their solving with Julia.


## 1. Introduction to Stochastic Linear Programming

A Stochastic Linear Program (LP) is an optimization problem where the *objective function is linear* (in the decision variables), and subject to *linear equality and inequality constraints* (of the decision variables) as for a linear program. However, that optimization problem is subject to uncertainty, commonly describe by a number of scenarios for the uncertain parameter $\omega$. An SLP can always be written as a minimization or as a maximization problem. And, in general, SLPs can have a mix of all types of constraints (equality and inequality). For simplicity and compactness, we will stick to their description in a standard minimization from, i.e., as minimization problems and with equality constraints only. It is always possible based on a number of dedicated tricks to reformulate a SLP into its standard form.

A key aspect of SLPs is that they include 2 types of decisions:
 - "here-and-now" decisions, which are to be taken at the first stage of the problem, and seen as binding,
 - "wait-and-see" decisions (also referred to as recourse), which are to be taken when uncertainty unfolds eventually
 
Here-and-now decisions are to be scenario-independent as they are supposed to hold whatever future realization of the uncertainty $\omega$. This aspect is also commonly referred to as a non-anticipativity constraint or property. In constrat, the recourse decisions can be different for different outcomes of the uncertainty $\omega$.

Let us formulate SLPs in their standard minimization form. 

Write $\mathbf{x}= [x_1, \, x_2, \, \ldots, \, x_n ]^\top$, $\mathbf{x} \in \mathbb{R}^n$, a vector of $n$ here-and now decision variables and $\mathbf{c}= [c_1, \, c_2, \, \ldots, \, c_n ]^\top$, $\mathbf{c} \in \mathbb{R}^n$, the unit cost vector associated to those variables. In parallel, define $A \in \mathbb{R}^{m_\text{h} \times n}$ the matrix gathering the coefficients for the $m_\text{h}$ linear equality constraints for the $n$ here-and-now decision variables, as well as $\mathbf{b} \in \mathbb{R}^{m_\text{h}}$ corresponding right-hand side. An SLP writes

$$
\begin{align}
\min_{\mathbf{x}} \quad  & \mathbf{c}^\top \mathbf{x} + \mathbb{E}_\omega \left[ Q(\mathbf{x}, \omega) \right] \nonumber\\
\text{s.t.} \quad & A \mathbf{x} = \mathbf{b} \nonumber \\
                  & \mathbf{x} \geq 0 \nonumber
\end{align}
$$

In the above, $\mathbb{E}_\omega \left[ Q(\mathbf{x}, \omega) \right]$ is the expected recourse cost where the recourse cost is defined as another minimization problem based on recourse decision variables $\mathbf{y}= [y_1, \, y_2, \, \ldots, \, y_l ]^\top$, $\mathbf{y} \in \mathbb{R}^l$, and related cost $\mathbf{d}= [d_1, \, d_2, \, \ldots, \, d_l ]^\top$, $\mathbf{d} \in \mathbb{R}^l$, i.e.,

$$
\begin{align} 
Q(\mathbf{x},\omega) \quad = \quad \min_{\mathbf{y}} \quad  & \mathbf{d}(\omega)^\top \mathbf{y} \nonumber\\
\text{s.t.} \quad & T(\omega) \mathbf{x} + W(\omega) \mathbf{y} = \mathbf{h}(\omega) \nonumber \\
                  & \mathbf{y} \geq 0 \nonumber
\end{align}
$$

This second linear program has constraints that bind the here-and-now decisions with the recourse decisions, with $T \in \mathbb{R}^{m_\text{r} \times n}$ the matrix of coefficients related to $\mathbf{x}$ (and for $m_\text{r}$ recourse constraints), $W \in \mathbb{R}^{m_\text{r} \times l}$ the matrix of coefficient related to $\mathbf{y}$ and $\mathbf{h}\in \mathbb{R}^{m_\text{r} \times l}$ the right-hand side. This second linear problem is conditioned to $\omega$ since, in principle, there may be a single best decision for $\mathbf{y}$ for any realization of that uncertainty.

An interesting aspect though is that, since dealing with a min-min, and since the expectation can be approximated based on a set of samples $\omega_k$ for the uncertainty ($k=1,\ldots,K$), an SLP like that in the above admits a deterministic equivalent as

$$
\begin{align}
\min_{\mathbf{x},\mathbf{y}} \quad  & \mathbf{c}^\top \mathbf{x} + \sum_{k=1}^K p_k \mathbf{d}_k^\top \mathbf{y}_k  \nonumber\\
\text{s.t.} \quad & A \mathbf{x} = \mathbf{b} \nonumber \\
                  & T_k \mathbf{x} + W_k \mathbf{y}_k = \mathbf{h}_k & k=1,\ldots,K \nonumber \\
                  & \mathbf{x} \geq 0 \nonumber \\
                  & \mathbf{y}_k \geq 0, & k=1,\ldots,K \nonumber
\end{align}
$$

The $p_k$'s are general weights for the various possible outcomes for $\omega$. If those are equally likely, one simply has $p_k=1/K$. The deterministic equivalent is also called the *extensive form*.

Note that in the special case where the matrix $W_k$ is not a function of $\omega_k$, we talk about a stochastic linear program with *fixed* recourse. To indicate that, we may use the notation $W$ instead. 



## 2. Application example 1: Gas procurement problem

This application example is inspired by many stylized examples available in textbooks and online (e.g. at neos-guide.org).

Gas has to be procured in advance, while the consumption of gas is highly linked to ambient temperature (since used for heating mostly). Gas retailer (and possibly system) operators have to procure gas in advance for direct use, and possibly also to be stored. They then rely on forecasts for future weather (e.g., "is next winter going to be a cold one, or not?") to be used as input to decision-making. Since predicting how cold the next year will be, there is quite some uncertainty in the gas demand. We want to make optimal decisions on gas procurement in view of such uncertainty.

In the following, we first go through the way the problem is set up and then reformulated as an SLP. Then, we show how to solve that problem in Julia. 

## 2.1 Problem formulation

Assume we currently have a "normal" winter, and we want to procure gas thinking of current winter, but also winter next year. A weather/climate forecaster tells you that next winter may be either "normal", "cold" and "freezing cold", with probabilities $p_1=$ 1/4, $p_2=$ 1/4 and $p_3=$ 1/2. The price of gas and the demand for it, are a direct function of how cold the winter may be. This is summarized in the following table:

| scenario $k$ | Winter type | Demand $\omega_k$(units) | Gas price $\pi_k$ (DKK/unit) |
| :- | :- | :-: | :-: |
| 1 | normal     | 60  | 60
| 2 | cold       | 100 | 70
| 3 | freezing   | 200 | 100


That means that, we know we need to procure 100 units of gas for the current year (as it is a "normal" winter) and then have to think of what we do about next year, under uncertainty on quantity and price. We have the possibility to store gas though, and it costs us $\pi_s$ 10 DKK/unit per year.

The first questions we need to ask ourselves for such problems are
- what is the uncertain parameter $\omega$
- what are our here-and-now decisions $\mathbf{x}$?
- what are the recourse decisions $\mathbf{y}$ (to be index by $k$ since dependent upon scenario $k$ for the uncertainty)

From the problem description, it is fairly clear that the uncertain parameter $\omega$ is the uncertain demand for gas in year 2. And, the demand for gas in year 1 is knows, since it is the current year and it is a "normal" one. In parallel, the course of actions seem to be as follows
- we have to decide what to buy and store now, to have enough for this year, and be ready for next year (this is happening, whatever the scenario for year 2),
- depending on the type of winter, we may have to buy the gas that we miss (this depends on the scenario considered)
Based on this description of the course of action, the first set of decisions are the here-and-now decisions $\mathbf{x} = [l^{(1)} \, \, s^{(1)}]^\top$, for what we buy and store in year 1. In parallel, the unique recourse decision is $y_k = l^{(2)}_k$, for what we will buy and withdraw from the storage at year 2, within scenario $k$. In the present case, we have $K=3$ scenarios, with $k=1$ for the "normal" one, $k=2$ for the "cold" one and $k=3$ for the "freezing" one.

Focusing directly on the extensive form, this eventually writes
$$
\begin{align} 
\min_{\mathbf{x},\mathbf{y}} \quad  & \mathbf{c}^\top \mathbf{x} + \sum_{k=1}^K p_k d_k^\top y_k  \nonumber\\
\text{s.t.} \quad & A \mathbf{x} = \mathbf{b} \nonumber \\
                  & T \mathbf{x} + W y_k = h_k & k=1,\ldots,K \nonumber \\
                  & \mathbf{x} \geq 0 \nonumber \\
                  & y_k \geq 0, & k=1,\ldots,K \nonumber
\end{align}
$$
where, for the first-stage decisions,
$$ A=\left[\begin{array}{cc} 1 & 0 \\ 0 & 0 \end{array}\right], \qquad b = \left[ \begin{array}{c} \omega_1\\0\end{array}\right] $$
and, for the second-stage decisions,
$$ T=\left[ 0 \, \, 1 \right], \qquad W=1, \qquad h_k = \omega_k $$
Finally, for the cost function,
$$ \mathbf{c} = \left[ \begin{array}{c} \pi_1 \\ \pi_s \end{array}\right], \qquad d_k = \pi_k $$

The is a stochastic linear program with fixed recourse, since both matrices $T$ and $W$ are independent of the uncertain parameter $\omega$ and its potential realizations $\omega_k$.





### 2.2 Solving in Julia with JuMP and GPLK


In [3]:
#Import necessary Julia packages
using LinearAlgebra
using JuMP
using GLPK

#Declare model and optimizer
gasmodel = Model()
set_optimizer(gasmodel, GLPK.Optimizer)

#Define parameters
pv = [0.25 0.25 0.5]'
ω = [60 100 200]'
πv = [60 70 100]'
ns = length(ω)
πs = 10
c = [πv[1] πs]'


#Define variables
@variable(gasmodel, x[1:2])
@variable(gasmodel, y[1:3])

#Define Constraints
@constraint(gasmodel, x[1] == ω[1])
@constraint(gasmodel, [k=1:ns], x[2] + y[k] == ω[k])
@constraint(gasmodel, [k=1:ns], y[k] >= 0)
@constraint(gasmodel, [k=1:2], x[k] >= 0)

#Define Objective
@objective(gasmodel, Min, sum(c.*x) + sum( (pv[k] * πv[k] * y[k]) for k in 1:ns))

#Run the opimization
optimize!(gasmodel)

In [4]:
value.(x)

2-element Vector{Float64}:
 60.0
 60.0

In [5]:
value.(y)

3-element Vector{Float64}:
   0.0
  40.0
 140.0

In [6]:
objective_value(gasmodel)

11900.0

The solution is fairly obvious: the purchase for the first year is equal to the demand, while the best decision in terms of storage is to maximize the amount stored (and purchased in the first year too). This is since the cost for storage is far less than the price difference between normal and colder years. However, since we implicitely consider that the storage needs to be emptied the second year, it puts a limit on how much is stored, which is the quantity that will be needed, in case of a normal year.

## 3. Application example 2: Newsvendor problem as a SLP (consumption side)

The newsvendor problem is a very common, and widely studied, problem in decision-making under uncertainty. It was one of the main focus points of a previous block of the course ("Stochastic optimization and the newsvendor problem"). While it was shown at the time that it may be possible to obtain a closed-form solution for the newsvendor problem (assuming that the uncertainty distribution is known or can be modelled), it can also be solved as a stochastic linear program, without any assumption about the uncertainty. It relies on scenarios instead. 

In the following, we first go through the way the problem is set up and then reformulated as an SLP. Then, we show how to solve that problem in Julia. This version of the problem is on the consumption side, in the sense that the decision-maker has to decide how many paper to buy (so, "consume") now, to be then sold further on. In the next example, we will look at the other type of noewsvendor problem, on the production side.


### 3.1 Problem setup and formulation

In the newsvendor problem, we first need to identify the here-and-now decisions, and the recourse decisions. 

Remember that, in the stylised version of that problem, the newsvendor has to decide in advance (say in the morning before to go sell them) on a quantity $x$ of newpapers to purchase, to then be sold on the street eventually, under uncertain demand. The newsvendor has a number $K$ of scenarios for that demand, which we denote by $\omega_k$, $k=1,\ldots,K$, and with probability $p_k$ ($\sum_k p_k = 1$). Hence, it appears that the here-and-now decision is that quantity $x$. The cost of purchasing the newspapers at this stage is $c$ (ex: $c=$ 10 DKK).

Whenever selling a newspaper, the newsvendor makes a profit of $\delta$ (ex: $\delta=$ 5 DKK), hence meaning that the newspapers are sold at a unit price $c+\delta$ (ex: $c+\delta=$ 15 DKK). For any unsold newspaper at the end of the day, the newsvendor will bring them back to the depot and get a refund of $\eta$ (ex: $\eta=$ 2 DKK) per unit of newspapers not sold. This translates to a direct loss of $c-\eta$ per unit not sold.

There are then 2 potential cases:
 - either demand is higher than the quantity of newspapers originally purchased, and the newsvendor incurs an opportunity loss of $\delta$ per unit that could have been sold
 - or, demand is less than the quantity of newspapers originally purchased, and the newvendor incurs a loss of $c+\eta$ per unit that was not sold

Based on these two cases, one could define the asymmetric loss function that was discussed when solving the newsvendor problem analytically. Looking at the problem as an SLP, these cases relate to a number of recourse decisions, i.e., the number of sold newspapers $y_1$, the number of missing newspapers $y_2$ (i.e., demand is there, but the newsvendor does not have enough of them), and the number of recycled newspapers $y_3$. Clearly, if $y_2=0$, then $y_3\geq0$, and if $y_3=0$, then $y_2\geq0$.

Let us then formulate this problem as an SLP, as

$$
\begin{align}
\min_{x} && c x + \mathbb{E}_\omega \left[ Q(x, \omega) \right]  \nonumber\\
&& Q(x,\omega) & = \min_{y_1,y_2,y_3} & - (c+\delta) y_1 -\eta y_3 \nonumber \\
&&             & \qquad \text{s.t.}          & y_1 + y_2 = \omega \nonumber \\
&&             &                      & y_1 + y_3 = x \nonumber \\
&&             &                      & x,y_1,y_2,y_3 \geq 0 \nonumber
\end{align}
$$

In practice, since we have a number of scenarios and associated probabilities, the SLP problem in its deterministic form becomes

$$
\begin{align} 
\min_{x} \quad     & c x - \sum_{k=1}^K p_k \left( (c+\delta) y_{1,k} + \eta y_{3,k} \right) \nonumber \\
\text{s.t.} \quad  & y_{1,k} + y_{2,k} = \omega_k, \quad k=1,\ldots,K \nonumber \\
                   & y_{1,k} + y_{3,k} = x,  \quad k=1,\ldots,K \nonumber \\
                   & x,y_{1,k},y_{2,k},y_{3,k} \geq 0 \nonumber
\end{align}
$$

Since the here-and-now decision is scenario-independent (non-anticipativity constraint), we only have one decision variable $x$ in the above problem. However, since the recourse variable $y_1$, $y_2$ and $y_3$ are scenario dependent, we have to find an optimal value for each and every scenario $\omega_k$. This is why an additional subscript $k$ appears in notations for those decision variables.

Finally, if aiming to write that problem in a format similar to what we saw in Section 1, the here-and-now constraints can be removed (i.e., related to $A$ and $\mathbf{b}$) as there are none. In parallel, one needs to write the recourse cost vector as

$$ \mathbf{d} = [-(c+\delta) \, \, 0 \, \, -\eta]^\top $$

the matrix $T$ and $W$ for the recourse constraints as

$$ T = \left( \begin{array}{c}
0\\
-1
\end{array}
\right)
$$

and 

$$ W = \left( \begin{array}{ccc}
1 & 1 & 0\\
1 & 0 & 1\\
\end{array}
\right)
$$

as well as the right-hand side $h$ as

$$ h_k = \left( \begin{array}{c}
\omega_k\\
0
\end{array}
\right)
$$

In the above, both $T$ and $W$ matrices are independent of $\omega_k$, hence we have a stochastic linear program with fixed recourse.



### 3.2 Solving in Julia with JuMP and GPLK



In [7]:
#Import necessary Julia packages
using LinearAlgebra
using JuMP
using GLPK

#Declare model and optimizer
nvmodel = Model()
set_optimizer(nvmodel, GLPK.Optimizer)

#Define parameters
ω = 1:99
ns = length(ω)
c = 1
δ = 0.5
η = 0.2

#Define variables
@variable(nvmodel, x)
@variable(nvmodel, y_1[1:ns])
@variable(nvmodel, y_2[1:ns])
@variable(nvmodel, y_3[1:ns])

#Define Constraints
@constraint(nvmodel, [i=1:ns], y_1[i] + y_2[i] == ω[i])
@constraint(nvmodel, [i=1:ns], - x + y_1[i] + y_3[i] == 0)
@constraint(nvmodel, [i=1:ns], y_1[i] >= 0)
@constraint(nvmodel, [i=1:ns], y_2[i] >= 0)
@constraint(nvmodel, [i=1:ns], y_3[i] >= 0)
@constraint(nvmodel, x >= 0)

#Define Objective
@objective(nvmodel, Min, c * x + (1/ns) * sum( (-(c+δ)*y_1[i] - η*y_3[i]) for i in 1:ns))

#Run the opimization
optimize!(nvmodel)

In [8]:
value.(x)

39.0

In [9]:
value.(y_1)

99-element Vector{Float64}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0
 11.0
 12.0
 13.0
  ⋮
 39.0
 39.0
 39.0
 39.0
 39.0
 39.0
 39.0
 39.0
 39.0
 39.0
 39.0
 39.0

In [10]:
value.(y_2)

99-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
 49.0
 50.0
 51.0
 52.0
 53.0
 54.0
 55.0
 56.0
 57.0
 58.0
 59.0
 60.0

In [11]:
value.(y_3)

99-element Vector{Float64}:
 38.0
 37.0
 36.0
 35.0
 34.0
 33.0
 32.0
 31.0
 30.0
 29.0
 28.0
 27.0
 26.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [12]:
objective_value(nvmodel)

-9.769696969696966

## 4. Application example 3: Newsvendor problem as a SLP (production side)

While the above version of the newsvendor problem looked at the consumption side (which is seen at the traditional version of the problem), we focus here on the symmetric problem if one is on the production side. This is for instance the case if selling renewable energy generation in electricity markets: one should place a bid in advance (the day before) while how much a wind farm or solar power plant will produce the day after cannot be known with certainty. However, if one produces more of less energy than in the contract, there is a penalty.

In the following, we first go through the way the problem is set up and then reformulated as an SLP. Then, we show how to solve that problem in Julia. 

### 4.1 Problem setup and formulation

As for the previous case, when starting with a newsvendor problem, we first need to identify the here-and-now decisions, and the recourse decisions. 

In the production version of the problem, a renewable energy trader (for instance) has to decide in advance (one day for the next) on a quantity $x$ of energy to sell on the electricity market. It is not the demand here that is uncertain, but how much the renewable energy asset will produce eventually. The trader has a number $K$ of scenarios for that production, which we denote by $\omega_k$, $k=1,\ldots,K$, and with probability $p_k$ ($\sum_k p_k = 1$). Hence, it appears that the here-and-now decision is that quantity $x$. The unit price received when selling the quantity $x$ on the electricity market is $c$ (ex: $c=$ 100 DKK/MWh).

Now, whenever the actual energy production is less than the contracted quantity $x$, the trader has to compensate by buying the quantity missing from another market at a price $c+\delta$, e.g. $\delta=$ 20 DKK/MWh. And, if producing more energy than the contracted quantity $x$, the market will take it, but at a price $c-\eta$ that is lower that original price $c$ on the electricity market (e.g., $\eta=$ 10 DKK/MWh). All in all, that means that, compared the case where the trader could perfectly predict how much will be produced, there are then 2 potential cases:
- if energy production is more than the contracted value $x$, there will be an opportunity cost of $\eta$ per unit of energy above $x$, 
- if energy production is less than the contracted value $x$, there will be a direct loss of $\delta$ for any unit of energy below $x$.

Based on these two cases, one could define the asymmetric loss function that was discussed when solving the newsvendor problem analytically. Looking at the problem as an SLP, these cases relate to a number of recourse decisions, i.e., the quantity of energy produced $y_1$, the quantity of energy missing $y_2$ (if production is less than the contract $x$), and the surplus of energy $y_3$ (in case the production is less than the contract $x$). Clearly, if $y_2=0$, then $y_3\geq0$, and if $y_3=0$, then $y_2\geq0$.

Normally, a trader would want to maximize revenue. However, for convenience, we want to write it as a minimization problem (hence, minus the revenue). Let us then formulate this problem as an SLP, as

$$
\begin{align}
\min_{x} && (-c) x + \mathbb{E}_\omega \left[ Q(x, \omega) \right] \nonumber\\
&& Q(x,\omega) & = \min_{y_1,y_2,y_3} & (c+\delta) y_2 - (c-\eta) y_3 \nonumber \\
&&             & \qquad \text{s.t.}          & y_1 + y_2 = x \nonumber \\
&&             &                      & y_1 + y_3 = \omega \nonumber \\
&&             &                      & x,y_1,y_2,y_3 \geq 0 \nonumber
\end{align}
$$

In practice, since we have a number of scenarios and associated probabilities, the SLP problem in its deterministic form becomes

$$
\begin{align}
\min_{x} \quad     & (-c) x + \sum_{k=1}^K p_k \left( (c+\delta) y_{2,k} - (c-\eta) y_{3,k} \right) \nonumber \\
\text{s.t.} \quad  & y_{1,k} + y_{2,k} = x, \quad k=1,\ldots,K \nonumber \\
                   & y_{1,k} + y_{3,k} = \omega_k,  \quad k=1,\ldots,K \nonumber \\
                   & x,y_{1,k},y_{2,k},y_{3,k} \geq 0 \nonumber
\end{align}
$$

Since the here-and-now decision is scenario-independent (non-anticipativity constraint), we only have one decision variable $x$ in the above problem. However, since the recourse variable $y_1$, $y_2$ and $y_3$ are scenario dependent, we have to find an optimal value for each and every scenario $\omega_k$. This is why an additional subscript $k$ appears in notations for those decision variables.

Finally, if aiming to write that problem in a format similar to what we obtained in Section 1, the here-and-now constraints can be removed (i.e., related to $A$ and $\mathbf{b}$) as there are none. In parallel, one needs to write the recourse cost vector as

$$ \mathbf{d} = [0 \, \,  (c+\delta) \, \, -(c-\eta)]^\top $$

the matrix $T$ and $W$ for the recourse constraints as

$$ T = \left( \begin{array}{c}
0\\
-1
\end{array}
\right)
$$

and 

$$ W = \left( \begin{array}{ccc}
1 & 1 & 0\\
1 & 0 & 1\\
\end{array}
\right)
$$

as well as the right-hand side $h$ as

$$ h_k = \left( \begin{array}{c}
0\\
\omega_k
\end{array}
\right)
$$

In the above, both $T$ and $W$ matrices are independent of $\omega_k$, hence we have a stochastic linear program with fixed recourse.


### 4.2 Solving in Julia with JuMP and GPLK

In [13]:
#Import necessary Julia packages
using LinearAlgebra
using JuMP
using GLPK

#Declare model and optimizer
nvmodel = Model()
set_optimizer(nvmodel, GLPK.Optimizer)

#Define parameters
ω = 1:99
ns = length(ω)
c = 100
δ = 20
η = 10

#Define variables
@variable(nvmodel, x)
@variable(nvmodel, y_1[1:ns])
@variable(nvmodel, y_2[1:ns])
@variable(nvmodel, y_3[1:ns])

#Define Constraints
@constraint(nvmodel, [i=1:ns], - x + y_1[i] + y_2[i] == 0)
@constraint(nvmodel, [i=1:ns], y_1[i] + y_3[i] == ω[i])
@constraint(nvmodel, [i=1:ns], y_1[i] >= 0)
@constraint(nvmodel, [i=1:ns], y_2[i] >= 0)
@constraint(nvmodel, [i=1:ns], y_3[i] >= 0)
@constraint(nvmodel, x >= 0)

#Define Objective
@objective(nvmodel, Min, (-c) * x + (1/ns) * sum( ((c+δ)*y_2[i] - (c-η)*y_3[i]) for i in 1:ns))

#Run the opimization
optimize!(nvmodel)

In [14]:
value.(x)

34.0

In [15]:
value.(y_1)

99-element Vector{Float64}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0
 11.0
 12.0
 13.0
  ⋮
 34.0
 34.0
 34.0
 34.0
 34.0
 34.0
 34.0
 34.0
 34.0
 34.0
 34.0
 34.0

In [16]:
value.(y_2)

99-element Vector{Float64}:
 33.0
 32.0
 31.0
 30.0
 29.0
 28.0
 27.0
 26.0
 25.0
 24.0
 23.0
 22.0
 21.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [17]:
value.(y_3)

99-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
 54.0
 55.0
 56.0
 57.0
 58.0
 59.0
 60.0
 61.0
 62.0
 63.0
 64.0
 65.0

In [18]:
objective_value(nvmodel)

-4670.0